# Machine Learning - Hw 2

# Multi-class Weather Classification


##Import libraries

In [0]:
%tensorflow_version 1.x

import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import warnings

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers

from keras import callbacks

warnings.filterwarnings('ignore')

print("Tensorflow version %s" %tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Load data

Get data from  https://drive.google.com/drive/folders/1UzH28Q8xki8_DMYdDgHxi40-CJ800Kaq




In [0]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

datadir = '/content/drive/My Drive/Hw2-ML/'
trainingset = datadir + '/MWI-Dataset-1.1_3200/'
testset = datadir + '/MWI-testset/'

batch_size = 32
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,\
    zoom_range=0.1,\
    rotation_range=45,\
    horizontal_flip=True,\
    vertical_flip=False)

train_generator = train_datagen.flow_from_directory(
    directory=trainingset,
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

test_datagen = ImageDataGenerator(
    rescale = 1. / 255)

test_generator = test_datagen.flow_from_directory(
    directory=testset,
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = [k for k,v in train_generator.class_indices.items()]

print("Image input %s" %str(input_shape))
print("Classes: %r" %classnames)

print('Loaded %d training samples from %d classes.' %(num_samples,num_classes))
print('Loaded %d test samples from %d classes.' %(test_generator.n,test_generator.num_classes))


##Show *n* random images


In [0]:
import matplotlib.pyplot as plt

n = 3
x,y = train_generator.next()
# x,y size is train_generator.batch_size

for i in range(0,n):
    image = x[i]
    label = y[i].argmax()  # categorical from one-hot-encoding
    print(classnames[label])
    plt.imshow(image)
    plt.show()
    
    


## LeNet

In [0]:
def LeNet(input_shape, num_classes, regl2 = 0.0001, lr=0.0001):  
    model = Sequential()
    
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=input_shape, padding="same"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
    
    model.add(Flatten())

    model.add(Dense(84, activation='tanh'))

    model.add(Dense(num_classes, activation='softmax'))

    adam = optimizers.Adam(lr=lr)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# create the model
model = LeNet(input_shape,num_classes)
model.summary()
  

##AlexNet

In [0]:
def AlexNet(input_shape, num_classes, regl2 = 0.0001, lr=0.0001):

    model = Sequential()

    # C1 Convolutional Layer 
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11),strides=(2,4), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # C2 Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # C3 Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # C4 Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # C5 Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Flatten
    model.add(Flatten())

    flatten_shape = (input_shape[0]*input_shape[1]*input_shape[2],)
    
    # D1 Dense Layer
    model.add(Dense(4096, input_shape=flatten_shape, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # D2 Dense Layer
    model.add(Dense(4096, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # D3 Dense Layer
    model.add(Dense(1000,kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # Compile

    adam = optimizers.Adam(lr=lr)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model
 
# create the model
model = AlexNet(input_shape,num_classes)
model.summary()

## ValerioNet 1

In [0]:
 def ValerioNet1(input_shape, num_classes): 
    model = Sequential()

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape=train_generator.image_shape))
    model.add(AveragePooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))


    model.add(Flatten())
    model.add(Dense(100, activation = "relu", ))
    model.add(Dropout(0.5))
    model.add(Dense(train_generator.num_classes, activation = "softmax"))

    optimizer = 'adam'
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

    return model

# create the model
model = ValerioNet1(input_shape,num_classes)
model.summary()


## ValerioNet 2

In [0]:
def ValerioNet2(input_shape, num_classes):    
    model = Sequential()

    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=input_shape, padding="same"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    
    model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    model.add(BatchNormalization())
        
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    
    model.add(Dense(500, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(train_generator.num_classes, activation = "softmax"))

    optimizer = 'adam'
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

    return model

# create the model
model = ValerioNet2(input_shape,num_classes)
model.summary()
  

## Train

In [0]:
#stopping = callbacks.EarlyStopping(monitor='val_acc', patience=3)
#callbacks=[stopping],

steps_per_epoch=train_generator.n//train_generator.batch_size
val_steps=test_generator.n//test_generator.batch_size+1

try:
    history = model.fit_generator(train_generator, epochs=100, verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=test_generator,\
                    validation_steps=val_steps)
except KeyboardInterrupt:
    pass


##Save the model

In [0]:
import os

models_dir = '/content/drive/My Drive/Hw2-ML/models/'

def savemodel(model,problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

# Save the model
savemodel(model,'Weather_classes_100_ValerioNet1_epochs_differentset_3')

## Load Pre-Trained Model

In [0]:
import os
from keras.models import load_model

models_dir = datadir + '/models/'

ValerioNet1 = 'MWI_ValerioNet1'
ValerioNet2 = 'MWI_ValerioNet2'
Tansfer_ValerioNet1 = 'MWI_ValerioNet1_Transfer'
Tansfer_ValerioNet2 = 'MWI_ValerioNet2_Transfer'

def loadmodel(problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    try:
        model = load_model(filename)
        print("\nModel loaded successfully from file %s\n" %filename)
    except OSError:    
        print("\nModel file %s not found!!!\n" %filename)
        model = None
    return model

model = loadmodel(ValerioNet2)

## MWI-Dataset Evaluation Scores

In [0]:
import sklearn.metrics 
from sklearn.metrics import classification_report, confusion_matrix

test_generator = test_datagen.flow_from_directory(
    directory=testset,
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

val_steps=(test_generator.n//test_generator.batch_size+1) 


print('############## ACCURACY ##############')
loss, acc = model.evaluate_generator(test_generator,verbose=1,steps=val_steps)
print('MWI-Dataset loss: %f' %loss)
print('MWI-Dataset accuracy: %f' %acc)
print()

preds = model.predict_generator(test_generator,verbose=1,steps=val_steps-1)

Ypred = np.argmax(preds, axis=1)
Ytest = test_generator.classes  # shuffle=False in test_generator

print('############## PRECISION, RECALL, F-SCORE ##############')
print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))

print('############## CONFUSION MATRIX ##############')
cm = confusion_matrix(Ytest, Ypred)

print(cm)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
  for j in range(0,cm.shape[1]):
    if (i!=j and cm[i][j]>0):
      conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')
for k in conf:
  print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/test_generator.n))

## SMART-I Evaluation Scores

In [0]:
import sklearn.metrics 
from sklearn.metrics import classification_report, confusion_matrix

smarti_test = datadir + '/SMART-I_Dataset'

test_generator = test_datagen.flow_from_directory(
    directory=smarti_test,
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

val_steps=test_generator.n//test_generator.batch_size+1


############## ACCURACY ##############
loss, acc = model.evaluate_generator(test_generator,verbose=1,steps=val_steps)
print('SMART-I loss: %f' %loss)
print('SMART-I accuracy: %f' %acc)

preds = model.predict_generator(test_generator,verbose=1,steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = test_generator.classes  # shuffle=False in test_generator

############## PRECISION, RECALL, F-SCORE ##############
print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))

############## CONFUSION MATRIX ##############

cm = confusion_matrix(Ytest, Ypred)

print(cm)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
  for j in range(0,cm.shape[1]):
    if (i!=j and cm[i][j]>0):
      conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')
for k in conf:
  print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/test_generator.n))